The purpose of this exercise is to demonstrate the use of RNNs using generated data. By generating data ourselves, we have more control over what we train and test the model on.


In [27]:
import keras as K
import numpy as np
import pandas as pd

from random import randint, random
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.recurrent import LSTM, SimpleRNN

from sklearn.metrics import confusion_matrix, f1_score

# Types of Cats

Cats are lazy. They do only one thing every hour. There exists several types of cats. 

Testing that RNN's are capable of performing sequence prediction over different kinds of sequences.

In [28]:
# Most cats perform one of four actions
EAT = 0
NAP = 1
SCRATCH = 2 
BEAT_UP_STINKY_DOG = 3

NUM_POSSIBLE_NORMAL_ACTIONS = 4
NUM_POSSIBLE_ACTIONS = 5

In [29]:
def next_action_round_robin(current_action): 
    return (current_action + 1) % NUM_POSSIBLE_NORMAL_ACTIONS

def random_action(): 
    return randint(0, NUM_POSSIBLE_NORMAL_ACTIONS - 1)

In [30]:
# Predictable Cat behaves in a round robin fashion. Robin... how ironic. 
def predictable_cat():
    action = random_action()
    while True: 
        yield action
        action = next_action_round_robin(action)

In [31]:
# OCD Cat does everything in threes, no more, no less...
def ocd_cat(): 
    action = random_action()
    while True: 
        yield action
        yield action
        yield action
        action = next_action_round_robin(action)

In [32]:
# Unpredictable cat performs a random action sometimes, and resumes the sequence from there
# This adds some noise to our data, the RNN seems robust to that based on accuracy score. 
# On hindsight, should have tagged the cats to check which ones get misclassified.
def unpredictable_cat():
    action = random_action()
    while True:
        action = next_action_round_robin(action) if random() < 0.8 else random_action()
        yield action

In [33]:
# Smelly Cat raids the dumpster every other action
RAID_DUMPSTER = 4

def smelly_cat():
    action = random_action()
    while True:
        yield action
        yield RAID_DUMPSTER
        action = next_action_round_robin(action)

In [34]:
# Let's have a look at Smelly Cat's day
cat = smelly_cat()
smelly_actions = [next(cat) for i in range(0,20)]

In [35]:
def cat_randomizer():
#     cat_gens = [predictable_cat, ocd_cat, smelly_cat]
    cat_gens = [predictable_cat, unpredictable_cat, ocd_cat, smelly_cat]

    while True:
        yield cat_gens[randint(0, 2)]()

In [36]:
def actions_for(cat, num_actions):
    return [next(cat) for i in range(0, num_actions)]

# Data Preparation

We will use one-hot encoding to represent actions.

In [37]:
def one_hot(labels, num_classes): 
    return [[1 if i == label else 0 for i in range(0, num_classes)] for label in labels]

In [38]:
one_hot(smelly_actions, NUM_POSSIBLE_ACTIONS)

[[1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1]]

And now for some kitty action!

In [39]:
def bring_me_some_cats(cat_count):
    rand_cat = cat_randomizer()
    for i in range(0, cat_count):
        yield next(rand_cat)
#         yield predictable_cat()

In [40]:
num_actions = 51
num_cats = 500
kitties_actions = [one_hot(actions_for(cat, num_actions), NUM_POSSIBLE_ACTIONS) for cat in bring_me_some_cats(num_cats)]

Splice out the last action, this is the "label" that we want to predict.

In [41]:
X = kitties_initial_actions = [k[:num_actions - 1] for k in kitties_actions]
y = kitties_last_action = [k[-1] for k in kitties_actions]

Split the data set into a training set and test set.

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

# Construct Model

## Model Hyperparameters

In [43]:
EPOCHS = 120
HIDDEN = 10

In [119]:
model = Sequential()
# model.add(LSTM(HIDDEN, input_shape=[50, 5]))
model.add(SimpleRNN(HIDDEN, input_shape=[50, 5]))
model.add(Dense(5, activation='softmax'))

#model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [120]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=100)

Epoch 1/120
375/375 [==============================] - 0s - loss: 1.7337 - acc: 0.1467     
Epoch 2/120
375/375 [==============================] - 0s - loss: 1.6893 - acc: 0.1893     
Epoch 3/120
375/375 [==============================] - 0s - loss: 1.6466 - acc: 0.3120     
Epoch 4/120
375/375 [==============================] - 0s - loss: 1.6017 - acc: 0.3200     
Epoch 5/120
375/375 [==============================] - 0s - loss: 1.5571 - acc: 0.3253     
Epoch 6/120
375/375 [==============================] - 0s - loss: 1.5123 - acc: 0.3547     
Epoch 7/120
375/375 [==============================] - 0s - loss: 1.4698 - acc: 0.4213     
Epoch 8/120
375/375 [==============================] - 0s - loss: 1.4294 - acc: 0.4907     
Epoch 9/120
375/375 [==============================] - 0s - loss: 1.3930 - acc: 0.5013     
Epoch 10/120
375/375 [==============================] - 0s - loss: 1.3594 - acc: 0.5120     
Epoch 11/120
375/375 [==============================] - 0s - loss: 1.3295 - acc

# Model Evaluation

In [121]:
def label_from_vector(v): 
    return np.argmax(v)

In [122]:
y_predicted = model.predict(X_test)

In [123]:
actual_labels = [label_from_vector(yt) for yt in y_test]
predicted_labels = [label_from_vector(yp) for yp in y_predicted]

In [124]:
confusion_matrix(actual_labels, predicted_labels)

array([[25,  2,  3,  1],
       [ 1, 29,  2,  0],
       [ 1,  1, 23,  2],
       [ 0,  1,  3, 31]])

In [125]:
f1_score(actual_labels, predicted_labels, average='macro')

0.86150770768461926

# Sometimes, cats' behaviors depend on the weather

The purpose of this section is (still unrealized) two-fold. 
1) Show the effectiveness on LSTMs on tasks that RNN's cannot handle (long sequences where associations are separated far in time),
2) Incorporate an input (weather) at each time step and in combination with past history, make a prediction.

Note: I have tried varying the data some ways, but have not yet found a case where LSTMs outperform simple RNN's. 

In [44]:
SUNNY = 0
RAINY = 1    

In [45]:
def sunny_rainy_sunny(rainy_period_start, rainy_duration):
    for i in range(0, rainy_period_start):
        yield SUNNY
    for i in range(0, rainy_duration):
        yield RAINY
    while True: 
        yield SUNNY

In [49]:
def rain_cat(weather):
    action = random_action()
    while True:
        if next(weather) == RAINY: 
            yield NAP
        else:
            yield action
            action = next_action_round_robin(action)

In [50]:
import functools

def random_weather_sequence():
    srs = sunny_rainy_sunny(randint(5, 15), randint(20, 30))
    return srs

rain_cats = [rain_cat(random_weather_sequence()) for i in range(250)]
rain_cats_actions = [one_hot(actions_for(cat, num_actions), NUM_POSSIBLE_ACTIONS) for cat in rain_cats]

In [51]:
rain_cats_actions

[[[0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0]],
 [[0, 0, 0, 1, 0],
  [1, 0, 0,

In [52]:
X = rain_cats_initial_actions = [k[:num_actions - 1] for k in rain_cats_actions]
y = rain_cats_last_action = [k[-1] for k in rain_cats_actions]

Split the data set into a training set and test set.

In [270]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [271]:
%pprint
print(X_train)
# %pprint

Pretty printing has been turned ON
[[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0]], [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0,

# Construct Model

## Model Hyperparameters

In [272]:
EPOCHS = 120
HIDDEN = 10

In [273]:
model = Sequential()
# model.add(LSTM(HIDDEN, input_shape=[50, 5]))
model.add(SimpleRNN(HIDDEN, input_shape=[50, 5]))
model.add(Dense(5, activation='softmax'))

#model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [274]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=100)

Epoch 1/120
187/187 [==============================] - 0s - loss: 1.4695 - acc: 0.4599     
Epoch 2/120
187/187 [==============================] - 0s - loss: 1.4387 - acc: 0.4599     
Epoch 3/120
187/187 [==============================] - 0s - loss: 1.4076 - acc: 0.4866     
Epoch 4/120
187/187 [==============================] - 0s - loss: 1.3778 - acc: 0.4866     
Epoch 5/120
187/187 [==============================] - 0s - loss: 1.3478 - acc: 0.4866     
Epoch 6/120
187/187 [==============================] - 0s - loss: 1.3185 - acc: 0.4866     
Epoch 7/120
187/187 [==============================] - 0s - loss: 1.2894 - acc: 0.4866     
Epoch 8/120
187/187 [==============================] - 0s - loss: 1.2611 - acc: 0.4866     
Epoch 9/120
187/187 [==============================] - 0s - loss: 1.2333 - acc: 0.5027     
Epoch 10/120
187/187 [==============================] - 0s - loss: 1.2062 - acc: 0.5775     
Epoch 11/120
187/187 [==============================] - 0s - loss: 1.1786 - acc

# Model Evaluation

In [275]:
def label_from_vector(v): 
    return np.argmax(v)

In [276]:
y_predicted = model.predict(X_test)

In [277]:
actual_labels = [label_from_vector(yt) for yt in y_test]
predicted_labels = [label_from_vector(yp) for yp in y_predicted]

In [278]:
confusion_matrix(actual_labels, predicted_labels)

array([[13,  0,  0,  0],
       [ 0, 19,  0,  0],
       [ 0,  0, 13,  0],
       [ 0,  0,  0, 18]])

In [279]:
f1_score(actual_labels, predicted_labels, average='macro')

1.0